# Ejercicios del tema de Regresión

*Hugo Díaz Díaz*(*hdiazdd00@estudiantes.unileon.es*)

*Correo profesional: hugo.didi.contacto@gmail.com*

---

<br>

<blockquote style="border-left: 4px solid #00ccff; padding: 0.5em;">
  <strong>💡 NOTA PARA EL PROFESORADO</strong> <br><br> <b>La entrega se realiza en un Jupyter Notebook ajeno al de los apuntes, primero por temas de ejecución y montar mis propios flujos de datos desde cero y segundo por llevar un seguimiento y publicar mis ejercicios en Github, al que prefiero subir contenido exclusivamente propio. Espero que no sea una molestia y si así lo fuera lo cambiaré para la próxima entrega.<br><br> Gracias.</b>
</blockquote>

<br>


## Preguntas opcionales:

### 1. ¿Qué algoritmo de entrenamiento de regresión lineal puede utilizar si dispone de un conjunto de entrenamiento con millones de características?

---

Con millones de características, entrenaría con alguna variante de Gradient Descent (SGD o Mini-Batch). La Ecuación Normal y la SVD se vuelven muy lentas cuando n crece mucho, mientras que GD escala bien con el número de características.

### 2. Suponga que las características de su conjunto de entrenamiento tienen escalas muy diferentes. ¿Qué algoritmos podrían verse afectados por esta situación? ¿Qué puede hacerse al respecto?

---

Si las variables están en escalas muy distintas, los algoritmos basados en Gradient Descent se ven afectados (convergencia muy lenta o inestable). La solución es escalar/estandarizar las características antes de entrenar (p. ej. con StandardScaler).

### 3. ¿Puede Gradient Descent quedarse atascado en un mínimo local al entrenar un modelo de Regresión Logística?

---

En regresión logística (modelo lineal) la función de pérdida (log-loss/entropía cruzada) es convexa respecto a los parámetros, por lo que no existen mínimos locales: cualquier mínimo es global. Por eso GD no suele “atascarse” en mínimos locales; además, la naturaleza estocástica de SGD ayuda a escapar de óptimos “malos” si los hubiera.

### 4. ¿Todos los algoritmos Gradient Descent conducen al mismo modelo si se les deja correr el tiempo suficiente?

---

Tras suficiente entrenamiento, los métodos de GD acaban en modelos muy similares. Batch GD se detiene en el mínimo; SGD y Mini-Batch se acercan y “oscilan” alrededor, pero con un buen schedule de learning rate también pueden asentarse.

### 5. Supongamos que utilizas Batch Gradient Descent y representas gráficamente el error de validación en cada epoch. Si observas que el error de validación aumenta constantemente, ¿qué puede estar pasando? ¿Cómo puede solucionarlo?

---

Si el error de validación sube en cada epoch con Batch GD, puede que estés sobreajustando o, más típicamente, que la tasa de aprendizaje sea demasiado alta y el entrenamiento esté divergendo. Solución: reduce η y/o aplica regularización o parada temprana para cortar cuando el error de validación deje de mejorar.


### 6. ¿Es una buena idea detener el Mini-Batch Gradient Descent inmediatamente cuando el error de validación aumenta?

---

No conviene parar Mini-Batch GD en cuanto sube una vez el error de validación: las curvas son ruidosas. Lo sensato es usar parada temprana con “paciencia” (esperar al mínimo claro) o suavizar la métrica; entonces cortar en el mínimo de validación.

### 7. ¿Qué algoritmo Gradient Descent (de entre los que hemos discutido) alcanzará más rápidamente la proximidad de la solución óptima? ¿Cuál convergerá realmente? ¿Cómo puedes hacer que los demás también converjan?

---

El que llega antes “cerca” del óptimo suele ser SGD (o Mini-Batch) porque cada actualización es barata. El que realmente converge y se queda en el mínimo es Batch GD; a los otros puedes hacerlos converger usando un schedule que reduzca gradualmente el learning rate.

### 8. Supongamos que se utiliza la regresión polinómica. Se trazan las curvas de aprendizaje y se observa que hay una gran diferencia entre el error de entrenamiento y el error de validación. ¿Qué ocurre? ¿Cuáles son las tres formas de resolverlo?

---

Si con regresión polinómica ves gran brecha entre error de entrenamiento (bajo) y de validación (alto), hay sobreajuste. Tres vías: reducir la complejidad (grado menor), añadir más datos de entrenamiento y/o regularizar (Ridge/Lasso/Elastic Net).

### 9. Supongamos que se utiliza la regresión Ridge y se observa que el error de entrenamiento y el error de validación son casi iguales y bastante elevados. ¿Diría que el modelo tiene un sesgo alto o una varianza alta? ¿Debería aumentar el hiperparámetro de regularización α o reducirlo?

---

Si en Ridge train y valid son casi iguales y altos, hay alto sesgo (underfitting). Para darle más capacidad, reduciría α (demasiada regularización aplana el modelo).

### 10. ¿Por qué querría utilizar...
- Regresión Ridge en lugar de Regresión Lineal simple (es decir, sin ninguna regularización)?
- Lasso en lugar de Ridge Regression?
- Elastic Net en lugar de Lasso?

---

Usaría Ridge en lugar de lineal “pura” para controlar el sobreajuste manteniendo pesos pequeños; Lasso en lugar de Ridge cuando prefiero penalización $l_1$; y Elastic Net cuando quiero un compromiso ajustable entre $l_1$ y $l_2$ (r mezcla ambos extremos).


### 11. Supongamos que se desea clasificar las imágenes como exteriores/interiores y diurnas/nocturnas. ¿Debería implementar dos clasificadores de Regresión Logística o un clasificador de Regresión Softmax?

---


Aquí son dos etiquetas independientes (exterior/interior y diurna/nocturna). Implementaría dos clasificadores logísticos binarios, no un Softmax, que está pensado para clases mutuamente excluyentes en un único eje multiclase. 

## Ejercicio obligatorio:

### 12. **(OBLIGATORIO)** Implementa Batch Gradient Descent con parada temprana para la Regresión Softmax (sin usar Scikit-Learn).